In [1]:
import time
import math
import re
from textblob import TextBlob
import pandas as pd

import nltk as nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import string


import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel

# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess
pyLDAvis.enable_notebook()

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

In [2]:
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

workers = num_processors-1

Available CPUs: 16


In [3]:
#! pip install pandarallel
import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [9]:
from google.cloud import storage
import os
import io
client = storage.Client()
bucket = client.get_bucket('nlp_final_data')

#blob = bucket.blob('data_no_stop.csv')
blob = bucket.blob('songs_even_sample.csv')
content = blob.download_as_string()

songs = pd.read_csv(io.BytesIO(content))

In [12]:
songs.head(5)

,Unnamed: 0,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,lyrics_cleaned
0,0,Hi-TEX,rap,Notty Taylor,2018,37,{},"[VERSE 1]\nKeep it 100 wit me\nCan't afford 50/50\n****** trap phone was jiggin'\nWe thought he ran the city\nYou on the way to money\nYou going hundred fifty\nWe counting 100s, 50s\nMy niggas getting jiggy\nMy niggas getting grimy\nThese niggas getting grimy\nThese niggas taking lives\nThese niggas shoot and fire\nWe gotta aim higher\nThese pigs tryna tame lions\nWe don't trust outsiders\nIn the summer we count bodies\nUmma smoke away all my problems\n\n[PRE-HOOK]\nFeel good, don't know\nWo...",4068265,en,en,en,"Keep it 100 wit me Can't afford 50/50 ****** trap phone was jiggin' We thought he ran the city You on the way to money You going hundred fifty We counting 100s, 50s My niggas getting jiggy My niggas getting grimy These niggas getting grimy These niggas taking lives These niggas shoot and fire We gotta aim higher These pigs tryna tame lions We don't trust outsiders In the summer we count bodies Umma smoke away all my problems Feel good, don't know Won't know, smoke more Get gone, go home H..."
1,1,Foes,rap,"Sk, the Novelist",2017,34,"{""Sk / the Novelist""}","[Intro]\nDevil creeping up on me, he's trying to get real close\nGod be on my side and we both be in battle pose\nThis the final lap and I don't really mean to boast\nBut, got the key to victory, you hitting sour notes\nDevil creeping up on me, he's trying to get real close\nGod be on my side and we both be in battle pose\nThis the final lap and I don't really mean to boast.....\n\n[Chorus]\nOkay, yeah\nI got love, love, love for my city\nI got love, love, love for my foes\nI got love, love,...",2974557,en,en,en,"Devil creeping up on me, he's trying to get real close God be on my side and we both be in battle pose This the final lap and I don't really mean to boast But, got the key to victory, you hitting sour notes Devil creeping up on me, he's trying to get real close God be on my side and we both be in battle pose This the final lap and I don't really mean to boast..... Okay, yeah I got love, love, love for my city I got love, love, love for my foes I got love, love, good people with me I got l..."
2,2,Omnitrix,rap,Coults,2021,52,{},"Intro:\nWoo! (Uhuh, uhuh, uhuh)\nWoo! (Uhuh, uhuh, uhuh)\nIf you thought I ain’t gonna talk my shit you were smoking on spice!\nHuh, Please refrain from playin wit me, I am not that guy!\n\nVerse 1:\nLiving a movie and I’m in the lead (uhuh)\nI’m a simple guy, I’ll pull up with 1,2,3,4,5,6 Gs\nI flooded my bitch, I flooded my wrist, what you know 'bout tsunamis\nMan got in his bag, what you know 'bout a bag, this bag can’t fit in my jeans\n\nChorus:\nAhh, You could call me Grandpa Max, Hawai...",7360310,en,en,en,"Intro: Woo! (Uhuh, uhuh, uhuh) Woo! (Uhuh, uhuh, uhuh) If you thought I ain’t gonna talk my shit you were smoking on spice! Huh, Please refrain from playin wit me, I am not that guy! Verse 1: Living a movie and I’m in the lead (uhuh) I’m a simple guy, I’ll pull up with 1,2,3,4,5,6 Gs I flooded my bitch, I flooded my wrist, what you know 'bout tsunamis Man got in his bag, what you know 'bout a bag, this bag can’t fit in my jeans Chorus: Ahh, You could call me Grandpa Max, Hawaiian clothes! ..."
3,3,Do What I Want,rap,Champagne937,2019,17,{},"“Really I’m a genius bro, I’m really and truly a genius, and niggas hate me\nThey hate on me cause I’m rich, I look better than them\nI probably fucked they bitch, it is what it is”\nI Love Champagne\nJETWAY\n\nYou got on designer I hope That its bulletproof\nIt’s Jetway nigga who is you\nI’m fucking ya bitch an I kept on my tennis shoes\nYeah I do what real niggas do\nMe and Cierra said fuck it an took a cruise\nBoat jumping like kangaroos\nHer friends is a Delta the other a Zeta\nTogether...",4981347,en,en,en,"“Really I’m a genius bro, I’m really and truly a genius, and niggas hate me They hate on me cause I’m rich, I look better than the

In [7]:
%%time

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

songs['lyrics_cleaned'] = songs['lyrics_cleaned'].map(lambda x: re.sub('[^a-zA-Z0-9 @ . , : - _]', '', str(x)))
data_list = songs['lyrics_cleaned'].tolist()

CPU times: user 8.82 s, sys: 65.8 ms, total: 8.89 s
Wall time: 8.88 s


In [8]:
data_tokens = list(sent_to_words(data_list))

In [12]:
songs['data_tokens'] = data_tokens

In [15]:
songs_tokens = songs[['title','tag','artist','data_tokens']]

In [16]:
songs_tokens.to_csv('songs_w_tokens.csv')

In [ ]:
bigram = gensim.models.Phrases(data_tokens, min_count=1, threshold=1)
trigram = gensim.models.Phrases(bigram[data_tokens], threshold=1)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [9]:
%%time

stop_words = stopwords.words('english')

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

CPU times: user 2.72 ms, sys: 3.84 ms, total: 6.56 ms
Wall time: 3.92 ms


In [10]:
# Remove Stop Words
data_tokens_nostops = remove_stopwords(list(df.data_tokens))

In [12]:
df['tokens_no_stop'] = data_tokens_nostops

In [14]:
df.drop(columns=['Unnamed: 0','data_tokens']).to_csv('data_no_stop.csv')

In [15]:
blob = bucket.blob('data_no_stop.csv')
blob.upload_from_filename('data_no_stop.csv')

In [ ]:
%%time
# Create n-grams
data_words_bigrams = make_bigrams(data_tokens_nostops)
data_words_trigrams = make_trigrams(data_tokens_nostops)

# Combine tokens and n-grams
data_tokens_cobnined = data_tokens_nostops + data_words_bigrams + data_words_trigrams
#data_tokens_cobnined = data_words_trigrams

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Lemmatize text keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_tokens_cobnined, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [ ]:
# assumptions. Heavily depend on the set of songs that we have for analysis and doesnt represent the entirity of the genre. 